# POC-Turtle-7: "Dumb Explosive Parser"
This is a continuation of proof-of-concept (POC) experiments in unsupervised language learning (ULL), the OpenCog project hosted on [GitHub](https://github.com/opencog/language-learning/tree/master/notebooks).   
Previous results are shared as [static html copies of Jupyter notebooks](http://88.99.210.144/data/clustering_2018/html/).

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
from src.utl.utl import UTC
from src.utl.turtle import html_table
print(UTC(), ':: module_path:', module_path)

2018-03-16 05:42:32 UTC :: module_path: /home/obaskov/language-learning


## 7.1 Settings, parameters, data.

In [2]:
prj_dir = '../output/Turtle-7-2018-03-15/'  # project directory 
prefix = ''     # all project files will start with this prefix
verbose = 'max' # printed comments: 'none', 'min', 'max'
log = {'project': 'POC-Turtle-7: Explosive Parser'}

if not os.path.exists(prj_dir):
    os.makedirs(prj_dir)
    print('Project directory created:', module_path + prj_dir[2:])
else: print('Project directory', module_path + prj_dir[2:], 'exists')
path = module_path + prj_dir[2:]        # path to store vectors
tmpath = path  # module_path + '/tmp/'  # path for temporary files

Project directory /home/obaskov/language-learning/output/Turtle-7-2018-03-15/ exists


In [3]:
input_file = '../data/poc-turtle-sentences.txt'
if os.path.isfile(input_file):
    print('Data file:', module_path + input_file[2:],)
    log.update({'input_file': module_path + input_file[2:]})
    if verbose == 'max':
        print('- "Turtle" language corpus:')
        with open(input_file, 'r') as f: 
            lines = f.read().splitlines() # [0:2]
        for i,line in enumerate(lines): 
            if len(line) > 0: print(str(i+1)+'. '+line)    
else: print('No data file', module_path + input_file[2:])

Data file: /home/obaskov/language-learning/data/poc-turtle-sentences.txt
- "Turtle" language corpus:
1. tuna isa fish.
2. herring isa fish.
3. tuna has fin.
4. herring has fin.
5. parrot isa bird.
6. eagle isa bird.
7. parrot has wing.
8. eagle has wing.
9. fin isa extremity.
10. wing isa extremity.
11. fin has scale.
12. wing has feather.


## 7.2 Parse "synthetic" disjuncts from sentences.
We create pseudo-disjuncts for each word in each sentence taking all af the sentence words as its left and right neighbour words.

In [4]:
def explosive_parser(input_file, lw='LEFT-WALL', dot=True, verbose='none'):  # 80316 Turtle-7
    djs = pd.DataFrame(columns=['word','disjunct','count'])
    i = 0
    with open(input_file, 'r') as f: sentences = f.readlines() # [0:1]
    for j,sentence in enumerate(sentences):
        if len(sentence) > 1:
            sentence = sentence.rstrip()
            if dot == True:
                if sentence[-2]== ' .': continue
                elif sentence[-1] == '.': sentence = sentence[:-1] + ' .'
                else : sentence += ' .'
            else:
                if sentence[-2] == ' .': sentence = sentence[:-2]
                elif sentence[-1] == '.': sentence = sentence[:-1]
            if type(lw) == str and lw != 'none' and lw != '':
                sentence = lw + ' ' + sentence
            words = sentence.split()
            for k,word in enumerate(words):
                if k == 0:
                    for l in range(k+1, len(words)):
                        disjunct = words[l] + '+'
                        count = 1.0/(l-k)
                        djs.loc[i] = [word, disjunct, count]
                        i += 1
                else:
                    for m in range(0, k):  # single left disjuncts
                        disjunct = words[m] + '-'
                        count = 1.0/(k-m)
                        djs.loc[i] = [word, disjunct, count]
                        i += 1
                    for n in range(k+1, len(words)):  # single right disjuncts
                        disjunct = words[n] + '+'
                        count = 1.0/(n-k)
                        djs.loc[i] = [word, disjunct, count]
                        i += 1
        #-elif verbose == 'max': print('Empty line - EOF?', input_file)
    return djs

left_wall = 'LEFT-WALL' # Left wall symbol, '', 'none' - don't use Left-Wall
parses = explosive_parser(input_file, left_wall, dot=True, verbose='max')
if verbose != 'none': print(len(parses),'disjuncts parsed from', input_file)

240 disjuncts parsed from ../data/poc-turtle-sentences.txt


In [5]:
parses.head(12) 
# parses # uncomment to display 240-row DataFrame

,word,disjunct,count
0,LEFT-WALL,tuna+,1.000000
1,LEFT-WALL,isa+,0.500000
2,LEFT-WALL,fish+,0.333333
3,LEFT-WALL,.+,0.250000
4,tuna,LEFT-WALL-,1.000000
5,tuna,isa+,1.000000
6,tuna,fish+,0.500000
7,tuna,.+,0.333333
8,isa,LEFT-WALL-,0.500000
9,isa,tuna-,1.000000


In [6]:
disjuncts = parses.groupby(['word','disjunct'], as_index=False).sum() \
    .sort_values(by=['count','word','disjunct'], ascending=[False,True,True]) \
    .reset_index(drop=True)
dj_number = len(set(disjuncts['disjunct'].tolist()))
word_number = len(set(disjuncts['word'].tolist()))
if verbose != 'none': print(word_number, 'words and', dj_number, \
    'unique disjuncts form', len(disjuncts),'unique word-disjunct pairs') 

15 words and 28 unique disjuncts form 116 unique word-disjunct pairs


In [7]:
disjuncts.head(12)
# disjuncts # uncomment to display 116-row DataFrame

,word,disjunct,count
0,.,LEFT-WALL-,3.000000
1,.,has-,3.000000
2,.,isa-,3.000000
3,LEFT-WALL,.+,3.000000
4,LEFT-WALL,has+,3.000000
5,LEFT-WALL,isa+,3.000000
6,has,.+,3.000000
7,has,LEFT-WALL-,3.000000
8,isa,.+,3.000000
9,isa,LEFT-WALL-,3.000000


## 7.3 Learn multi-germ-multi-disjunct lexical entries
We cluster words and disjuncts to form unique multi-germ-multi-disjunct lexical entries, following the Link Grammar rules. Actually the entries are low-level Link Grammar rules, but Link Grammar would not accept word-based disjuncts.

In [8]:
from src.link_grammar.turtle import entries2clusters, lexical_entries
stalks = entries2clusters(lexical_entries(disjuncts))
stalks

,germs,disjuncts,counts,cluster
C01,[.],"[LEFT-WALL-, bird-, eagle-, extremity-, feathe...",25.000000,C01
C02,[LEFT-WALL],"[.+, bird+, eagle+, extremity+, feather+, fin+...",25.000000,C02
C03,[bird],"[.+, LEFT-WALL-, eagle-, isa-, parrot-]",5.666667,C03
C04,"[eagle, parrot]","[.+, LEFT-WALL-, bird+, has+, isa+, wing+]",11.333333,C04
C05,[extremity],"[.+, LEFT-WALL-, fin-, isa-, wing-]",5.666667,C05
C06,[feather],"[.+, LEFT-WALL-, has-, wing-]",2.833333,C06
C07,[fin],"[.+, LEFT-WALL-, extremity+, has+, has-, herri...",11.333333,C07
C08,[fish],"[.+, LEFT-WALL-, herring-, isa-, tuna-]",5.666667,C08
C09,[has],"[.+, LEFT-WALL-, eagle-, feather+, fin+, fin-,...",18.000000,C09
C10,"[herring, tuna]","[.+, LEFT-WALL-, fin+, fish+, has+, isa+]",11.333333,C10


In [9]:
# Display word-based Link Grammar rules (not valid for LG parser)
# from src.link_grammar.turtle import entries2rules
# display(html_table([['Cluster','Germs','','','Disjuncts']] + entries2rules(stalks)))

## 7.4 Learn Link Grammar rules from multi-germ-multi-disjunct lexical entries
The next step from multi-germ-multi-disjunct lexical entries to Link Grammar rules is replacing words in disjuncts with cluster identificators.

In [10]:
from src.link_grammar.turtle import disjuncts2clusters
rules = disjuncts2clusters(stalks)  # DataFrame
rules

,germs,disjuncts,counts,cluster
C01,[.],"[C02C01-, C03C01-, C04C01-, C05C01-, C06C01-, ...",25.000000,C01
C02,[LEFT-WALL],"[C02C01+, C02C03+, C02C04+, C02C05+, C02C06+, ...",25.000000,C02
C03,[bird],"[C02C03-, C03C01+, C04C03-, C11C03-]",5.666667,C03
C04,"[eagle, parrot]","[C02C04-, C04C01+, C04C03+, C04C09+, C04C11+, ...",11.333333,C04
C05,[extremity],"[C02C05-, C05C01+, C07C05-, C11C05-, C13C05-]",5.666667,C05
C06,[feather],"[C02C06-, C06C01+, C09C06-, C13C06-]",2.833333,C06
C07,[fin],"[C02C07-, C07C01+, C07C05+, C07C09+, C07C11+, ...",11.333333,C07
C08,[fish],"[C02C08-, C08C01+, C10C08-, C11C08-]",5.666667,C08
C09,[has],"[C02C09-, C04C09-, C07C09-, C09C01+, C09C06+, ...",18.000000,C09
C10,"[herring, tuna]","[C02C10-, C10C01+, C10C07+, C10C08+, C10C09+, ...",11.333333,C10


In [11]:
from src.link_grammar.turtle import entries2rules
lg_rule_list = entries2rules(rules)
display(html_table([['Cluster','Germs','L','R','Disjuncts']] + lg_rule_list))

Cluster,Germs,L,R,Disjuncts
C01,['.'],[],[],"['C02C01-', 'C03C01-', 'C04C01-', 'C05C01-', 'C06C01-', 'C07C01-', 'C08C01-', 'C09C01-', 'C10C01-', 'C11C01-', 'C12C01-', 'C13C01-']"
C02,['LEFT-WALL'],[],[],"['C02C01+', 'C02C03+', 'C02C04+', 'C02C05+', 'C02C06+', 'C02C07+', 'C02C08+', 'C02C09+', 'C02C10+', 'C02C11+', 'C02C12+', 'C02C13+']"
C03,['bird'],[],[],"['C02C03-', 'C03C01+', 'C04C03-', 'C11C03-']"
C04,"['eagle', 'parrot']",[],[],"['C02C04-', 'C04C01+', 'C04C03+', 'C04C09+', 'C04C11+', 'C04C13+']"
C05,['extremity'],[],[],"['C02C05-', 'C05C01+', 'C07C05-', 'C11C05-', 'C13C05-']"
C06,['feather'],[],[],"['C02C06-', 'C06C01+', 'C09C06-', 'C13C06-']"
C07,['fin'],[],[],"['C02C07-', 'C07C01+', 'C07C05+', 'C07C09+', 'C07C11+', 'C07C12+', 'C09C07-', 'C10C07-']"
C08,['fish'],[],[],"['C02C08-', 'C08C01+', 'C10C08-', 'C11C08-']"
C09,['has'],[],[],"['C02C09-', 'C04C09-', 'C07C09-', 'C09C01+', 'C09C06+', 'C09C07+', 'C09C12+', 'C09C13+', 'C10C09-', 'C13C09-']"
C10,"['herring', 'tuna']",[],[],"['C02C10-', 'C10C01+', 'C10C07+', 'C10C08+', 'C10C09+', 'C10C11+']"


## 7.5 Save Link Grammar dictionary

In [12]:
from src.link_grammar.turtle import save_link_grammar
lg_file_string = save_link_grammar(lg_rule_list, path)
for line in lg_file_string.splitlines(): print(line)

% POC Turtle Link Grammar v.0.7 2018-03-16 05:42:33 UTC
<dictionary-version-number>: V0v0v7+;
<dictionary-locale>: EN4us+;

% C01
".":
(C02C01-) or (C03C01-) or (C04C01-) or (C05C01-) or (C06C01-) or (C07C01-) or (C08C01-) or (C09C01-) or (C10C01-) or (C11C01-) or (C12C01-) or (C13C01-);

% C02
"LEFT-WALL":
(C02C01+) or (C02C03+) or (C02C04+) or (C02C05+) or (C02C06+) or (C02C07+) or (C02C08+) or (C02C09+) or (C02C10+) or (C02C11+) or (C02C12+) or (C02C13+);

% C03
"bird":
(C02C03-) or (C03C01+) or (C04C03-) or (C11C03-);

% C04
"eagle" "parrot":
(C02C04-) or (C04C01+) or (C04C03+) or (C04C09+) or (C04C11+) or (C04C13+);

% C05
"extremity":
(C02C05-) or (C05C01+) or (C07C05-) or (C11C05-) or (C13C05-);

% C06
"feather":
(C02C06-) or (C06C01+) or (C09C06-) or (C13C06-);

% C07
"fin":
(C02C07-) or (C07C01+) or (C07C05+) or (C07C09+) or (C07C11+) or (C07C12+) or (C09C07-) or (C10C07-);

% C08
"fish":
(C02C08-) or (C08C01+) or (C10C08-) or (C11C08-);

% C09
"has":
(C02C09-) or (C04C09-) or

## 5.6 Test the dictionary with Link Grammar parser
The dictionary was tested with an external CLI Pyton API to the Link Grammar parser on the server. The code is in early beta. 
```
tuna isa fish.
`/home/obaskov/data/lg/poc-turtle_13C_2018-03-15_0007: Found 4 linkages (4 had no P.P. violations)`

    +-C02C10+            
    |       |            
LEFT-WALL tuna [isa] [fish.] 


    +----C02C11---+      
    |             |      
LEFT-WALL [tuna] isa [fish.] 


    +-------C02C08-------+    
    |                    |    
LEFT-WALL [tuna] [isa] fish [.] 


    +----------C02C01---------+
    |                         |
LEFT-WALL [tuna] [isa] [fish] . 



herring isa fish. : Found 4 linkages (4 had no P.P. violations)

    +-C02C10-+              
    |        |              
LEFT-WALL herring [isa] [fish.] 


    +-----C02C11-----+      
    |                |      
LEFT-WALL [herring] isa [fish.] 


    +---------C02C08--------+    
    |                       |    
LEFT-WALL [herring] [isa] fish [.] 


    +-----------C02C01-----------+
    |                            |
LEFT-WALL [herring] [isa] [fish] . 



tuna has fin. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C10+            
    |       |            
LEFT-WALL tuna [has] [fin.] 


    +----C02C09---+      
    |             |      
LEFT-WALL [tuna] has [fin.] 


    +-------C02C07------+    
    |                   |    
LEFT-WALL [tuna] [has] fin [.] 


    +---------C02C01---------+
    |                        |
LEFT-WALL [tuna] [has] [fin] . 



herring has fin. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C10-+              
    |        |              
LEFT-WALL herring [has] [fin.] 


    +-----C02C09-----+      
    |                |      
LEFT-WALL [herring] has [fin.] 


    +--------C02C07--------+    
    |                      |    
LEFT-WALL [herring] [has] fin [.] 


    +-----------C02C01----------+
    |                           |
LEFT-WALL [herring] [has] [fin] . 


parrot isa bird. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C04-+             
    |        |             
LEFT-WALL parrot [isa] [bird.] 


    +-----C02C11----+      
    |               |      
LEFT-WALL [parrot] isa [bird.] 


    +--------C02C03--------+    
    |                      |    
LEFT-WALL [parrot] [isa] bird [.] 


    +-----------C02C01----------+
    |                           |
LEFT-WALL [parrot] [isa] [bird] . 


eagle isa bird. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C04+             
    |       |             
LEFT-WALL eagle [isa] [bird.] 


    +----C02C11----+      
    |              |      
LEFT-WALL [eagle] isa [bird.] 


    +--------C02C03-------+    
    |                     |    
LEFT-WALL [eagle] [isa] bird [.] 


    +----------C02C01----------+
    |                          |
LEFT-WALL [eagle] [isa] [bird] . 


parrot has wing. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C04-+             
    |        |             
LEFT-WALL parrot [has] [wing.] 


    +-----C02C09----+      
    |               |      
LEFT-WALL [parrot] has [wing.] 


    +--------C02C13--------+    
    |                      |    
LEFT-WALL [parrot] [has] wing [.] 


    +-----------C02C01----------+
    |                           |
LEFT-WALL [parrot] [has] [wing] . 



eagle has wing. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C04+             
    |       |             
LEFT-WALL eagle [has] [wing.] 


    +----C02C09----+      
    |              |      
LEFT-WALL [eagle] has [wing.] 


    +--------C02C13-------+    
    |                     |    
LEFT-WALL [eagle] [has] wing [.] 


    +----------C02C01----------+
    |                          |
LEFT-WALL [eagle] [has] [wing] . 



fin isa extremity. : Found 4 linkages (4 had no P.P. violations)`

    +C02C07+               
    |      |               
LEFT-WALL fin [isa] [extremity.] 


    +---C02C11---+         
    |            |         
LEFT-WALL [fin] isa [extremity.] 


    +--------C02C05-------+       
    |                     |       
LEFT-WALL [fin] [isa] extremity [.] 


    +------------C02C01-----------+
    |                             |
LEFT-WALL [fin] [isa] [extremity] . 


wing isa extremity. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C13+               
    |       |               
LEFT-WALL wing [isa] [extremity.] 


    +----C02C11---+         
    |             |         
LEFT-WALL [wing] isa [extremity.] 


    +--------C02C05--------+       
    |                      |       
LEFT-WALL [wing] [isa] extremity [.] 


    +------------C02C01------------+
    |                              |
LEFT-WALL [wing] [isa] [extremity] . 



fin has scale. : Found 4 linkages (4 had no P.P. violations)`

    +C02C07+             
    |      |             
LEFT-WALL fin [has] [scale.] 


    +---C02C09---+       
    |            |       
LEFT-WALL [fin] has [scale.] 


    +-------C02C12------+     
    |                   |     
LEFT-WALL [fin] [has] scale [.] 


    +----------C02C01---------+
    |                         |
LEFT-WALL [fin] [has] [scale] . 



wing has feather. : Found 4 linkages (4 had no P.P. violations)`

    +-C02C13+              
    |       |              
LEFT-WALL wing [has] [feather.] 


    +----C02C09---+        
    |             |        
LEFT-WALL [wing] has [feather.] 


    +--------C02C06-------+      
    |                     |      
LEFT-WALL [wing] [has] feather [.] 


    +-----------C02C01-----------+
    |                            |
LEFT-WALL [wing] [has] [feather] . 
```

_P.S. Looks like a total fail..._

## 7.7 Link Grammar for a dataset with removed punctuation
Let's collect the grammar learning pipeline in a short summary and test it on the same dataset parsed without ###LEFT-WALL### and period.

In [13]:
def pipeline(input_file, left_wall='', period=False, verbose='none'):
    parses = explosive_parser(input_file, left_wall, dot=False, verbose='max')
    disjuncts = parses.groupby(['word','disjunct'], as_index=False).sum() \
        .sort_values(by=['count','word','disjunct'], ascending=[False,True,True]) \
        .reset_index(drop=True)
    dj_number = len(set(disjuncts['disjunct'].tolist()))
    if verbose != 'none': print(dj_number, 'unique disjuncts form', \
        len(disjuncts),'unique word-disjunct pairs from', len(parses), 'parsed items') 
    dfg = lexical_entries(disjuncts)
    dfc = entries2clusters(dfg)
    rules = disjuncts2clusters(dfc)
    lg_rule_list = entries2rules(rules)
    return lg_rule_list

lg_rule_list = pipeline(input_file, left_wall='', period=False, verbose='max')
display(html_table([['Cluster','Germs','','','Disjuncts']] + lg_rule_list))

17 unique disjuncts form 62 unique word-disjunct pairs from 72 parsed items


Cluster,Germs,,,Disjuncts
C01,['bird'],[],[],"['C02C01-', 'C09C01-']"
C02,"['eagle', 'parrot']",[],[],"['C02C01+', 'C02C07+', 'C02C09+', 'C02C11+']"
C03,['extremity'],[],[],"['C05C03-', 'C09C03-', 'C11C03-']"
C04,['feather'],[],[],"['C07C04-', 'C11C04-']"
C05,['fin'],[],[],"['C05C03+', 'C05C07+', 'C05C09+', 'C05C10+', 'C07C05-', 'C08C05-']"
C06,['fish'],[],[],"['C08C06-', 'C09C06-']"
C07,['has'],[],[],"['C02C07-', 'C05C07-', 'C07C04+', 'C07C05+', 'C07C10+', 'C07C11+', 'C08C07-', 'C11C07-']"
C08,"['herring', 'tuna']",[],[],"['C08C05+', 'C08C06+', 'C08C07+', 'C08C09+']"
C09,['isa'],[],[],"['C02C09-', 'C05C09-', 'C08C09-', 'C09C01+', 'C09C03+', 'C09C06+', 'C11C09-']"
C10,['scale'],[],[],"['C05C10-', 'C07C10-']"


In [14]:
lg_file_string = save_link_grammar(lg_rule_list, path)
for line in lg_file_string.splitlines(): print(line)

% POC Turtle Link Grammar v.0.7 2018-03-16 05:42:33 UTC
<dictionary-version-number>: V0v0v7+;
<dictionary-locale>: EN4us+;

% C01
"bird":
(C02C01-) or (C09C01-);

% C02
"eagle" "parrot":
(C02C01+) or (C02C07+) or (C02C09+) or (C02C11+);

% C03
"extremity":
(C05C03-) or (C09C03-) or (C11C03-);

% C04
"feather":
(C07C04-) or (C11C04-);

% C05
"fin":
(C05C03+) or (C05C07+) or (C05C09+) or (C05C10+) or (C07C05-) or (C08C05-);

% C06
"fish":
(C08C06-) or (C09C06-);

% C07
"has":
(C02C07-) or (C05C07-) or (C07C04+) or (C07C05+) or (C07C10+) or (C07C11+) or (C08C07-) or (C11C07-);

% C08
"herring" "tuna":
(C08C05+) or (C08C06+) or (C08C07+) or (C08C09+);

% C09
"isa":
(C02C09-) or (C05C09-) or (C08C09-) or (C09C01+) or (C09C03+) or (C09C06+) or (C11C09-);

% C10
"scale":
(C05C10-) or (C07C10-);

% C11
"wing":
(C02C11-) or (C07C11-) or (C11C03+) or (C11C04+) or (C11C07+) or (C11C09+);

UNKNOWN-WORD: XXX+;

% 11 word clusters, 11 Link Grammar rules.
% Link Grammar file saved to: /home/obaskov/l

### Link Grammar parsing tests with the learned 11-rule dictionary
```
tuna isa fish : Found 2 linkages (2 had no P.P. violations)`

  +C08C09+      
  |      |      
tuna    isa [fish] 


  +--C08C06--+
  |          |
tuna [isa] fish 


herring isa fish : Found 2 linkages (2 had no P.P. violations)`

   +C08C09+      
   |      |      
herring  isa [fish] 


   +---C08C06---+
   |            |
herring [isa] fish 


tuna has fin : Found 2 linkages (2 had no P.P. violations)`

  +C08C07+     
  |      |     
tuna    has [fin] 


  +--C08C05-+
  |         |
tuna [has] fin 


herring has fin : Found 2 linkages (2 had no P.P. violations)`

   +C08C07+     
   |      |     
herring  has [fin] 


   +---C08C05--+
   |           |
herring [has] fin 


parrot isa bird : Found 2 linkages (2 had no P.P. violations)`

   +C02C09+      
   |      |      
parrot   isa [bird] 


   +---C02C01--+
   |           |
parrot [isa] bird 


eagle isa bird : Found 2 linkages (2 had no P.P. violations)`

  +C02C09+      
  |      |      
eagle   isa [bird] 


  +---C02C01--+
  |           |
eagle [isa] bird 


parrot has wing : Found 2 linkages (2 had no P.P. violations)`

   +C02C07+      
   |      |      
parrot   has [wing] 


   +---C02C11--+
   |           |
parrot [has] wing 


eagle has wing : Found 2 linkages (2 had no P.P. violations)`

  +C02C07+      
  |      |      
eagle   has [wing] 


  +---C02C11--+
  |           |
eagle [has] wing 


fin isa extremity : Found 2 linkages (2 had no P.P. violations)`

 +C05C09+        
 |      |        
fin    isa [extremity] 


 +---C05C03---+
 |            |
fin [isa] extremity 


wing isa extremity : Found 2 linkages (2 had no P.P. violations)`

  +C11C09+        
  |      |        
wing    isa [extremity] 


  +---C11C03---+
  |            |
wing [isa] extremity 


fin has scale : Found 2 linkages (2 had no P.P. violations)`

 +C05C07+      
 |      |      
fin    has [scale] 


 +--C05C10--+
 |          |
fin [has] scale 


wing has feather : Found 2 linkages (2 had no P.P. violations)`

  +C11C07+       
  |      |       
wing    has [feather] 


  +---C11C04--+
  |           |
wing [has] feather 
```

## Resume
The first attempt to learn grammar with "dumb explosive parser" failed.

The Link Grammar dictionaries are shared via [http://88.99.210.144/data/clustering_2018/POC-Turtle-7-2018-03-15/](http://88.99.210.144/data/clustering_2018/POC-Turtle-7-2018-03-15/)